<a href="https://colab.research.google.com/github/sharmatec/Hello-World/blob/master/1098969_1dconv_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
dataset = pd.read_csv('/content/housing.csv')
print("Here are the first five rows of the dataset:")
dataset = dataset.dropna()

X = dataset.loc[:,'longitude' : 'median_income']
Y = dataset['median_house_value']

x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=2003)
x_train_np=x_train.to_numpy()
x_test_np=x_test.to_numpy()
y_train_np=y_train.to_numpy()
y_test_np=y_test.to_numpy()

dataset.head(10)

Here are the first five rows of the dataset:


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


In [0]:
import matplotlib.pyplot as plt
dataset.plot(subplots=True, figsize=(7,7)); 
plt.legend(loc='center')

In [0]:
import torch 
from torch.nn import Conv1d
from torch.nn import MaxPool1d
from torch.nn import Flatten
from torch.nn import Linear
from torch.nn.functional import relu
from torch.utils.data import DataLoader,TensorDataset

In [0]:
class CnnRegressor(torch.nn.Module):

  def __init__(self, batch_size, inputs, outputs):
    super(CnnRegressor,self).__init__()
    self.batch_size=batch_size
    self.inputs=inputs
    self.outputs=outputs
    self.input_layer=Conv1d(inputs,batch_size,1)
    self.max_pooling_layer=MaxPool1d(1)
    self.conv_layer=Conv1d(batch_size,128,1)
    self.flatten_layer=Flatten()
    self.linear_layer=Linear(128,64)
    self.output_layer=Linear(64,outputs)

  def feed(self,input):

    input=input.reshape((self.batch_size,self.inputs,1))

    output=relu(self.input_layer(input))

    output=self.max_pooling_layer(output)

    output=relu(self.conv_layer(output))
    
    output=self.flatten_layer(output)

    output=self.linear_layer(output)

    output=self.output_layer(output)
    return output

In [0]:
from torch.optim import SGD
from torch.nn import L1Loss
!pip install pytorch-ignite
from ignite.contrib.metrics.regression.r2_score import R2Score

In [0]:
batch_size=32
model = CnnRegressor(batch_size, X.shape[1],1)
model.cuda()

CnnRegressor(
  (input_layer): Conv1d(8, 32, kernel_size=(1,), stride=(1,))
  (max_pooling_layer): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv_layer): Conv1d(32, 128, kernel_size=(1,), stride=(1,))
  (flatten_layer): Flatten()
  (linear_layer): Linear(in_features=128, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)

In [0]:
def model_loss(model,dataset,train =False,optimizer =None):
  performance = L1Loss()
  score_metric =R2Score()

  avg_loss=0
  avg_score=0
  count=0
  for input,output in iter(dataset):

    predictions=model.feed(input)
    loss=performance(predictions,output)

    score_metric.update([predictions,output])
    score=score_metric.compute()

    if(train):
      optimizer.zero_grad()

      loss.backward()
      optimizer.step()

    avg_loss +=loss.item()
    avg_score +=score
    count += 1
  return avg_loss / count, avg_score / count

In [0]:
epochs=1600

optimizer=SGD(model.parameters(),lr=1e-5)

inputs=torch.from_numpy(x_train_np).cuda().float()
ouputs=torch.from_numpy(y_train_np.reshape(y_train_np.shape[0],1)).cuda().float()

tensor=TensorDataset(inputs,ouputs)
loader=DataLoader(tensor,batch_size,shuffle=True,drop_last=True)

for epoch in range(epochs):

  avg_loss,avg_r2_score=model_loss(model,loader,train=True,optimizer=optimizer)

  print("Epoch"+ str(epoch+1)+ ":\n\tLoss ="+ str(avg_loss)+ "\n\tR^2 Score=" + str (avg_r2_score))

Epoch1:
	Loss =68442.0919107343
	R^2 Score=0.3750714637282797
Epoch2:
	Loss =68159.27306964686
	R^2 Score=0.3904102548749563
Epoch3:
	Loss =68941.995235426
	R^2 Score=0.3877982527412739
Epoch4:
	Loss =69011.55312850336
	R^2 Score=0.38714595808735797
Epoch5:
	Loss =68385.77800938902
	R^2 Score=0.375202084739972
Epoch6:
	Loss =68316.75767236546
	R^2 Score=0.37252830600424547
Epoch7:
	Loss =68518.4550868834
	R^2 Score=0.38027172752983124
Epoch8:
	Loss =68592.08220641816
	R^2 Score=0.3687106175546664
Epoch9:
	Loss =68613.34040428812
	R^2 Score=0.3616565465115956
Epoch10:
	Loss =69004.68646650785
	R^2 Score=0.38515961833167534
Epoch11:
	Loss =69143.67367923206
	R^2 Score=0.35526733072490735
Epoch12:
	Loss =68197.38572029148
	R^2 Score=0.3818138428293841
Epoch13:
	Loss =68609.16921244394
	R^2 Score=0.368260624075333
Epoch14:
	Loss =68260.35440022421
	R^2 Score=0.3658884092864435
Epoch15:
	Loss =67935.34975826794
	R^2 Score=0.3785770975085485
Epoch16:
	Loss =67937.25539517937
	R^2 Score=0.391

In [0]:
inputs=torch.from_numpy(x_test_np).cuda().float()
outputs=torch.from_numpy(y_test_np.reshape(y_test_np.shape[0],1)).cuda().float()

tensor=TensorDataset(inputs,outputs)
loader=DataLoader(tensor,batch_size,shuffle=True,drop_last=True)

print(loader)
avg_loss_test,avg_r2_score_test = model_loss(model,loader,train=False,optimizer=None)
print("The model'sL1lossis:"+str(avg_loss_test))
print("The model'sR^2scoreis:"+str(avg_r2_score_test))

The model'sL1lossis:72592.68038651315
The model'sR^2scoreis:0.36456381153037315


In [0]:
torch.save(model,"/content/1098969_id.pthg")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type CnnRegressor. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
